In [14]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from email.mime.text import MIMEText
import base64
import email
import datetime

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

In [15]:
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)


    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

In [16]:
service = build('gmail', 'v1', credentials=creds)

In [17]:
def get_all_mails(service, label):
    results = service.users().messages().list(
        userId='me', labelIds=[label]).execute()
    messages = results.get('messages', [])
    while 'nextPageToken' in results:
        page_token = results['nextPageToken']
        results = service.users().messages().list(userId='me', labelIds=[
            label], pageToken=page_token).execute()
        messages.extend(results['messages'])
    return messages

In [18]:
def get_message(service, user_id, msg_id):
    message = service.users().messages().get(
        userId=user_id, id=msg_id, format='raw').execute()
    data_object = {}
    data_object['labelIds'] = " ".join(message['labelIds'])
    data_object['snippet'] = message['snippet']
    data_object['sizeEstimate'] = message['sizeEstimate']
    data_object['internalDate'] = datetime.datetime.fromtimestamp(
        int(message['internalDate'])/1000)
    
    message_ascii = base64.urlsafe_b64decode(message['raw'].encode('ASCII'))

    mime_msg = email.message_from_bytes(message_ascii)

    data_object['subject'] = mime_msg['Subject']

    for part in mime_msg.walk():
        if part.get_content_type() == 'text/plain':
            body = part.get_payload(decode=True)
            try:
                # convert to string
                data_object['body'] = body.decode('utf-8')
            except:
                data_object['body'] = body

    return data_object

In [19]:
labels = ['CHAT', 'TRASH', 'SPAM', 'CATEGORY_FORUMS','CATEGORY_UPDATES', 'CATEGORY_PROMOTIONS', 'CATEGORY_SOCIAL', 'UNREAD']

all_messages = []
for label in labels:
    # get all the mails with label = label
    all_messages += get_all_mails(service, label)

In [20]:
id_set = set()
for i in all_messages:
    id_set.add(i['id'])

In [21]:
all_messages = []

for i in id_set:
    all_messages.append(get_message(service=service, user_id="me", msg_id=i))

In [22]:
import pandas as pd

# data in the above format
data = {'labelIds': ['UNREAD', 'IMPORTANT', 'CATEGORY_FORUMS', 'TRASH'], 'snippet': '', 'sizeEstimate': 567443, 'internalDate': datetime.datetime(
    2023, 1, 12, 6, 58, 54), 'subject': 'Congratulations Team VIT - DESH Department for Receiving Best Paper Award', 'body': '[image: image.png]\r\n'}

df = pd.DataFrame(columns=data.keys())

# Creating multiple similar elements
for i in all_messages:
    df = df.append(i, ignore_index=True)

# name of the CSV file
filename = "data.csv"

# writing to CSV file
df.to_csv(filename, index=False)

/tmp/ipykernel_38655/3840555999.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(i, ignore_index=True)
/tmp/ipykernel_38655/3840555999.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(i, ignore_index=True)
/tmp/ipykernel_38655/3840555999.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(i, ignore_index=True)
/tmp/ipykernel_38655/3840555999.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(i, ignore_index=True)
/tmp/ipykernel_38655/3840555999.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.